In [ ]:
import os

import pandas as pd

#from simple_cysh import *

os.chdir(r'Z:\ChiPrivate\Chicago Reports and Evaluation\SY18\Eval Management\ACM_analysis')

In [ ]:
# Use email column as key
staff_df = get_cysh_df('Staff__c', ['Id', 'Name', 'Email__c', 'Organization__c', 'Role__c'], "Site__c = 'Chicago'", rename_id=True)
staff_df.rename(columns={'Organization__c':'School__c', 'Email__c':'Email'}, inplace=True)
staff_df.loc[:, 'Email'] = staff_df['Email'].str.lower()
roles = ['Senior Corps Team Leader', 'Team Leader', 'Corps Member', 'Second Year Corps Member']
staff_df = staff_df.loc[staff_df['Role__c'].isin(roles)]

school_df = get_cysh_df('Account', ['Id', 'Name'])
school_df.rename(columns={'Id':'School__c', 'Name':'School_Name'}, inplace=True)

In [ ]:
acm_df = staff_df.merge(school_df, on='School__c')

# Onboarding Survey

In [ ]:
acm_q0surv = pd.read_excel('SY18 FINAL Team Placements.xlsx')
acm_q0surv_cols = [
    'Full.Name (CYSH Staff Object)',
    'Tutoring.Experience.Months',
    #'Teaching.Credential',
    'Tutoring.Preference',
    'Math.Confidence',
    'Age',
    'Educational.Attainment'
]

acm_df = acm_df.merge(acm_q0surv[acm_q0surv_cols], how='left', left_on='Name', right_on='Full.Name (CYSH Staff Object)')
del acm_df['Full.Name (CYSH Staff Object)']

# Quarterly Surveys

In [ ]:
def get_survey_df(Q):
    """ Read in survey's data dictionary and raw survey results.
    
    Replaces coded values with categorical values and replaces likert score categorical values with numerical values.
    """
    
    data_dict = pd.read_excel('Raw Survey Data/FY18 ACM Survey Variable Info.xlsx', sheet_name=f'FY18 {Q} ACM', header=None)
    
    table_2_index = data_dict.loc[data_dict[0]=='Value', :].index.values[0]
    
    var_info = data_dict.iloc[2:(table_2_index-3), :]
    var_info.columns = data_dict.iloc[1, :]
    
    var_vals = data_dict.iloc[(table_2_index + 1):, 0:3]
    var_vals.columns=['Variable', 'Value', 'Label']
    var_vals.loc[:, 'Variable'] = var_vals['Variable'].ffill()
    
    survey_df = pd.read_csv(f'Raw Survey Data/FY18 {Q} ACM Survey.csv')
    survey_df.rename(columns={'Vemail':'Email'}, inplace=True)
    survey_df.loc[:, 'Email'] = survey_df['Email'].str.lower()
    
    for variable in var_vals['Variable'].unique():
        df = var_vals.loc[var_vals['Variable']==variable, :].copy()
        survey_df.loc[:, variable] = survey_df[variable].astype(str)
        survey_df.loc[:, variable] = survey_df[variable].map(dict(zip(df['Value'], df['Label'])))
    
    # There may be items with 3-point likert scales that are erroneously replaced in this step
    likert5_map = {'Strongly disagree': 1,
                   'Disagree': 2,
                   'Neither disagree or agree': 3,
                   'Agree': 4,
                   'Strongly agree': 5}

    survey_df = survey_df.replace(likert5_map)
    
    return survey_df

def select_cols(survey_df, Q):
    cols_dict = {
        'Q1':[
            'var5', # "I am pleased with my decision to join City Year"
            'var7', # "I intend to complete this year of service"
            'var31', # "How likely are you to recommend City Year to a"
            'var76', # "I understand the significance of being on a team in order for my service to be successful"
            'var77', # "I benefit from being a member of my school team  "
            'var79', # "I feel comfortable supporting students with Literacy"
            'var80', # "I feel comfortable supporting students with Math"
            'var85', # "I feel welcomed at the school I serve"
            'var86', # "I feel prepared to serve the students at this school"
            'var87', # "I am looking forward to working with my partner teacher this year"
            'var88', # "I feel equipped to build a strong relationship with my partner teacher"
            #'Q1NetPromoter'
        ],
        'Q2':[
            'var51O96', # Commute:What has been the most challenging part of your service year so far? Please select all that apply  
        ],
        'Q3':[
            'var31', 
            'var32', 
        ]
    }
    
    cols = cols_dict[Q]
    
    survey_df = survey_df[['Email'] + cols.copy()]
    
    rename_cols_map = dict(zip(cols, [f'{Q}_' + col for col in cols])
    
    survey_df.rename(columns=rename_cols_map), inplace=True)
    
    return survey_df

In [ ]:
acm_q1surv = get_survey_df(Q='Q1')
acm_q2surv = get_survey_df(Q='Q2')
acm_q3surv = get_survey_df(Q='Q3')
acm_q4surv = get_survey_df(Q='Q4')

In [ ]:
acm_q1surv = select_cols(acm_q1surv, 'Q1')
acm_q2surv = select_cols(acm_q2surv, 'Q2')
acm_q3surv = select_cols(acm_q3surv, 'Q3')

In [ ]:
acm_df = acm_df.merge(acm_q1surv, how='left', on='Email')
acm_df = acm_df.merge(acm_q2surv, how='left', on='Email')
acm_df = acm_df.merge(acm_q3surv, how='left', on='Email')
acm_df.to_csv('ACM_surveys.csv', index=False)

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', 4):
#     print(var_info_q1surv[['Variable', 'Label']])

In [ ]:
# Q2_survey = pd.read_excel('privatedata/FY18 Q2 ACM Survey Data - Chicago.xlsx')
# Q2_survey['EmployeeID'] = Q2_survey['EmployeeID'].astype(str)
# df = df.merge(Q2_survey, left_on='ID__c', right_on='EmployeeID')
# df.to_csv('ACM_Q2_survey.csv', index=False)